In [1]:
import wbgapi as wb
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import Dropdown, VBox, Output

world = gpd.read_file("ne_110m_admin_0_countries.shp")

world = world.rename(columns={'name': 'country_name'})

dbs_df = pd.DataFrame([(db['id'], db['name']) for db in wb.source.list()], columns=['id', 'name'])
db_dropdown = Dropdown(options=[(name, _id) for _id, name in zip(dbs_df['id'], dbs_df['name'])], description='Database:')

ind_dropdown = Dropdown(options=[], description='Indicator:')

output = Output()

def update_indicators(change):
    db_id = change['new']
    wb.db = db_id
    indicators = [(ind['id'], ind.get('value', '[No Description]')) for ind in wb.series.list()]
    indicators.sort(key=lambda x: x[1])
    ind_dropdown.options = [(name, _id) for _id, name in indicators]

db_dropdown.observe(update_indicators, names='value')

def visualize(indicator_id, database_id):
    with output:
        output.clear_output()
        wb.db = database_id

        df = wb.data.DataFrame(indicator_id, labels=True)
        df.reset_index(inplace=True)

        year_columns = [col for col in df.columns if col.startswith("YR")]
        df_long = df.melt(id_vars=["economy", "Country"], value_vars=year_columns,
                          var_name="year", value_name="value")
        df_long['year'] = df_long['year'].str.replace("YR", "").astype(int)
        df_long.dropna(subset=['value'], inplace=True)

        merged = world.merge(df_long.rename(columns={'economy': 'ISO_A3'}), how='left', on='ISO_A3')

        latest_year = df_long['year'].max()
        print(f"Showing static map for latest available year: {latest_year}")
        merged_latest = merged[merged['year'] == latest_year]

        fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        merged_latest.plot(column='value', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
        ax.set_title(f"{indicator_id} ({latest_year})", fontsize=16)
        plt.axis('off')
        

        min_val = merged['value'].min()
        max_val = merged['value'].max()

        fig2 = px.choropleth(
            merged,
            locations="ISO_A3",
            color="value",
            hover_name="Country",
            animation_frame="year",
            color_continuous_scale="Viridis",
            range_color=(min_val, max_val),    # fix color scale here
            title=f"{indicator_id} Over Time"
            )

        fig2.update_layout(
            width=900,
            height=600,
            geo=dict(showframe=False, showcoastlines=False),
            margin={"r":0,"t":30,"l":0,"b":0}
            )

        fig2.show()

def on_indicator_change(change):
    if change['new']:
        visualize(change['new'], db_dropdown.value)

ind_dropdown.observe(on_indicator_change, names='value')

update_indicators({'new': db_dropdown.value})

VBox([db_dropdown, ind_dropdown, output])